In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import misc as ms

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [2]:
reload(dl)
reload(ms)

<module 'fitter.misc' from '../fitter\misc.pyc'>

### Compare fits

In [3]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info(whose='mine')
other_results = data_loader.get_fit_info(whose='others')

In [4]:
filename = 'fit_comparison'
xlabel = '$F^\pm_K/F^\pm_\pi$'
fig = ms.plot_comparison('fit', fit_results, other_results=other_results, xlabel=xlabel, show_model_avg=True)
data_loader.save_plots(fig, output_filename=filename)

Done.


### Compare delta_su2 corrections

In [5]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info(whose='mine')
other_results = data_loader.get_fit_info(whose='others')

In [6]:
filename = 'delta-su2_comparison'
xlabel = '$\delta_{SU(2)}$'
fig = ms.plot_comparison('delta_su2', fit_results, xlabel=xlabel, show_model_avg=True)
data_loader.save_plots(fig, output_filename=filename)

Done.


### Compare LECs

In [7]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info(whose='mine')

In [8]:
lecs = ['L_4', 'L_5', # nlo terms
             'A_a', 'A_k', 'A_p', # nnlo terms
             'A_aa', 'A_ak', 'A_ap', # nnnlo terms
             'A_kk', 'A_kp', 'A_pp'] # more nnnlo terms

figs = []
for lec in lecs:
    xlabel = lec
    if lec in ['L_4', 'L_5']:
        show_model_avg = True
    else: 
        show_model_avg = False
        
    fig = ms.plot_comparison(lec, fit_results, xlabel=xlabel, show_model_avg=show_model_avg)
    figs.append(fig)
    
filename = 'lec_comparison'
data_loader.save_plots(figs, output_filename=filename)

Done.
